# Part 1: Arithmetic and More

Although TensorFlow is designed for training neural networks, it can be used almost like any other programming language, with support for general arithmetic, control flow, and more. This offers a eaiser way to get started in TensorFlow without getting caught up in the theory of machine learning. 

This set of tutorials assumes that you are already familiar with Python. Familiarity with NumPy is a plus, but not a requirement.

## Basics

Let's get started! First, import TensorFlow.

In [1]:
import tensorflow as tf

When you declare a variable, constant, or operation in TensorFlow, it gets added to a graph in the background as a node. **The node's value is not computed immediately when it is declared.** Instead, it gets it value when the graph is "executed".

In the following example, we make a constant that holds some text. But when we try to print it, we see that it's just a node that doesn't hold a value yet.

In [2]:
hello = tf.constant("Hello TensorFlow!")
print hello

Tensor("Const:0", shape=(), dtype=string)


We'll get what's a Tensor in a moment. First, how can we actually get our text to print?

**A TensorFlow Session allocates memory and allows nodes in a graph to be executed or evaluated.** In this case, we evaluate the constant node `hello` and its value is returned.

In [3]:
sess = tf.Session()
print sess.run(hello)

Hello TensorFlow!


## Arithmetic

Arithmetic is the core of TensorFlow. The next example computes $3 \times 4 + 5 = 17$. Note that with TensorFlow, some operators like `+` and `*` are overloaded. **But even when you use operators like `+` and `*`, TensorFlow is still creating nodes in the graph.**

Think about what should happen when we try to evaluate `a4` or `a5`. It needs `tf.multiply(a1, a2)` and `a3` to be evaluated first. After those happen, we get $17$.

In [4]:
a1 = tf.constant(3)
a2 = tf.constant(4)
a3 = tf.constant(5)
a4 = tf.add(tf.multiply(a1, a2), a3)
a5 = a1 * a2 + a3

print sess.run(a4)
print sess.run(a5)

17
17


As mentioned earlier, **Tensors are n-dimensional arrays that can be passed between operations in a graph.** It's the same idea as an `ndarray` in NumPy. Previously, our Tensors were rank 0, or scalars. In the following example, we'll use Tensors of rank 1 (vectors) and Tensors of rank 2 (matrices). As you can guess, rank is the number of dimensions in a Tensor. The shape of a Tensor is a list containing its size in each dimension.

You might be wondering why TensorFlow uses a graph and computes everything at the end, instead of when nodes are declared. One reason is so that there can be placeholders - Tensors that can be operated on just like constant Tensors, but their value is unknown until specified when evaluating. However, you must specify the datatype of a placeholder when declaring it, and while not necessary, it's often good practice to specify the shape too. Here, we let `x` be a placeholder.

**Note that \* refers to element-wise multiplication**, so we need to use `tf.matmul` for matrix multiplication. Also, we need to force `W` and `b` to floats in order to match the datatype of `x`.

In [5]:
x = tf.placeholder(tf.float32, shape=[3, 2])
W = tf.constant([[2, 0], [0, 2]], tf.float32)
b = tf.constant([0, 5], tf.float32)
result = tf.matmul(x, W) + b

print x
print W
print b
print result

Tensor("Placeholder:0", shape=(3, 2), dtype=float32)
Tensor("Const_4:0", shape=(2, 2), dtype=float32)
Tensor("Const_5:0", shape=(2,), dtype=float32)
Tensor("add_1:0", shape=(3, 2), dtype=float32)


The operations we just declared computes

$$x \begin{bmatrix} 2 & 0 \\ 0 & 2 \end{bmatrix} + \begin{bmatrix} 0 & 5 \end{bmatrix}$$

where $\begin{bmatrix} 0 & 5 \end{bmatrix}$ is broadcast to every row, [much like in NumPy](https://docs.scipy.org/doc/numpy-1.12.0/user/basics.broadcasting.html). **We can pass a value to `x` by specifying it in a dictionary.** When something is executed, `x` takes the specified value if needed. Then, `result` returns the following as expected.

$$\begin{bmatrix} 1 & 1 \\ 2 & 1 \\ 1 & 2 \end{bmatrix} \begin{bmatrix} 2 & 0 \\ 0 & 2 \end{bmatrix} + \begin{bmatrix} 0 & 5 \end{bmatrix} = \begin{bmatrix} 2 & 7 \\ 4 & 7 \\ 2 & 9 \end{bmatrix}$$

In [6]:
print sess.run(result, {x: [[1, 1], [2, 1], [1, 2]]})

[[ 2.  7.]
 [ 4.  7.]
 [ 2.  9.]]


For reference, TensorFlow also has `tf.subtract` (overloaded `-`), `tf.divide` (overloaded `/`), `tf.mod` (overloaded `%`), `tf.pow` (overloaded `**`), `tf.sqrt`, `tf.sin`, `tf.cos`, `tf.tan`, and much more that can be found on [the TensorFlow documentation](https://www.tensorflow.org/api_guides/python/math_ops).

## Basic Control Flow

Finally, I wanted to touch on how to manage control flow in TensorFlow. Although these operations are used far less than arithmetic in neural network training, they demonstrate how you can really do anything you want in TensorFlow. 

Let's start with what is essentially the if statement, which works like the ternary operator in many programming languages. The syntax is:

```python
tf.cond(condition, func_if_true, func_if_false)
```

In [7]:
condition1 = tf.placeholder(tf.bool)
f1 = lambda: tf.constant("The condition was indeed true.")
f2 = lambda: tf.constant("The condition... it wasn't true!")
sentence1 = tf.cond(condition1, f1, f2)

print sess.run(sentence1, {condition1: True})
print sess.run(sentence1, {condition1: False})

The condition was indeed true.
The condition... it wasn't true!


Importantly, **`tf.cond` requires the condition be a boolean Tensor**, not just a Python bool. Instead of specifying a condition directly, we can also use something like `tf.equal` to condition based on something else. Here, we check if the given input is equal to 2.5.

In [8]:
val = tf.placeholder(tf.float32)
const = tf.constant(2.5)

condition2 = tf.equal(val, const)

sentence2 = tf.cond(condition2, f1, f2)

print sess.run(sentence2, {val: 2.5})
print sess.run(sentence2, {val: 3.5})

The condition was indeed true.
The condition... it wasn't true!


TensorFlow also supports the other comparison operators you're familiar with, namely ```tf.not_equal```, ```tf.less```, ```tf.less_equal```, ```tf.greater```, and ```tf.greater_equal```. **While ```tf.equal``` and ```tf.not_equal``` do not support operator overloading, the other four do.**

```tf.equal``` and its cousins also have a second use that doesn't relate much to control flow. When used to compare non-scalar Tensors, ```tf.equal``` returns a Tensor that is ```True``` in every location where the two are equal and ```False``` elsewhere. For example, comparing ```[1, 2]``` with ```[1, 1]``` would return ```[True, False]```. As we'll see quite soon in the next part, this can be very useful.

## Wrap-up

Remember to **close the Session when you're done** to free up resources.

In [9]:
sess.close()

In this part, we learned about the following:

* The TensorFlow Graph, nodes, and Session
```python
Session.run(node_to_evaluate, optional: placeholder_dict)
Session.close()
```
* Tensor rank, usage, and broadcasting
```python
tf.constant(value, optional: datatype)
tf.placeholder(datatype, optional: shape)
```
* Arithmetic operations
```python
tf.add(a, b)
tf.multiply(a, b)
tf.matmul(a, b)
```
* Basic control flow
```python
tf.cond(condition, func_if_true, func_if_false)
tf.equal(a, b)
```
* Operator overloading

We're now ready to start training machine learning models with TensorFlow!